# Projeto Python IA: Inteligência Artificial e Previsões

### Case: Score de Crédito dos Clientes

Você foi contratado por um banco para conseguir definir o score de crédito dos clientes. Você precisa analisar todos os clientes do banco e, com base nessa análise, criar um modelo que consiga ler as informações dos novos clientes e dizer automaticamente o score de crédito deles.

In [21]:
#importa o banco de dados
import pandas as pd

tabela = pd.read_csv("clientes.csv")


In [22]:
#verifica se tem valores vazios ou em formato indevido
print(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

In [23]:
#transforma as colunas do tipo texto (object) em números
from sklearn.preprocessing import OrdinalEncoder

colunas_categoricas = [col for col in tabela.columns if tabela[col].dtype == "object" and col != "score_credito"]

codificador = OrdinalEncoder()

tabela[colunas_categoricas] = codificador.fit_transform(tabela[colunas_categoricas])

#verifica se as colunas foram alteradas para não serem mais texto (object)
print(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  float64
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  float64
 13  divida_total              100000 non-null  fl

In [24]:
#Escolhendo quais as colunas vão ser usadas para treinar os modelos
# y é a coluna que queremos que o modelo calcule, nesse caso a score_credito
# x são todas as colunas que serão usadas para prever o score_credito, lembrando de retirar ela e qualquer outra inútil

y = tabela["score_credito"]
x = tabela.drop(columns=["score_credito", "id_cliente"])

#separar os dados em treino e teste
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x ,y , test_size=0.3)


In [ ]:
#escolhendo os modelos
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#criando os modelos
modelo_arvore = RandomForestClassifier()
modelo_knn = KNeighborsClassifier()

#treinando os modelos
modelo_arvore.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)

In [26]:
#escolher o melhor modelo
previsao_arvore = modelo_arvore.predict(x_teste)
previsao_knn = modelo_knn.predict(x_teste)

#Acurácia
from sklearn.metrics import accuracy_score

display(accuracy_score(y_teste, previsao_arvore))
display(accuracy_score(y_teste, previsao_knn))

0.8274666666666667

0.7342333333333333

In [27]:
#O modelo de árvore tem 82% de acerto, então usaremos ele para fazer as previsões dos novos clientes

tabela_nova = pd.read_csv("novos_clientes.csv")

tabela_nova[colunas_categoricas] = codificador.transform(tabela_nova[colunas_categoricas])

previsoes = modelo_arvore.predict(tabela_nova)

display(previsoes)

#exportando resultado
tabela_nova["score_credito"] = previsoes

tabela_nova.to_csv("novos_clientes_com_score.csv", index=False)

array(['Poor', 'Poor', 'Standard'], dtype=object)